# Import libraries and helpers

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys

In [ ]:
sys.path.append('./helpers_models/')

In [ ]:
sys.path.append('./data_visualization_and_augmentations/')

In [ ]:
sys.path.append('../torch_videovision/')

In [ ]:
sys.path.append('../video-classification/ResNetCRNN/')

In [ ]:
sys.path.append('./important_csvs/')

In [ ]:
from helpers_lstm import *

In [ ]:
plt.rcParams['figure.figsize'] = (20,10)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 20}
plt.rc('font', **font)

# Load training and validation sets

In [ ]:
tensor_transform = get_tensor_transform('ImageNet')

In [ ]:
train_transform = get_video_transform(2)
valid_transform = get_video_transform(0)

In [ ]:
df = pd.read_csv('./important_csvs/events_with_number_of_frames_stratified.csv')
df = get_df(df, 16, False)

In [ ]:
class_image_paths, end_idx = get_indices(df)

In [ ]:
train_loader = get_loader(16, 4, end_idx, class_image_paths, train_transform, tensor_transform, True)

In [ ]:
# show_batch(train_loader,4)

In [ ]:
df = pd.read_csv('./important_csvs/events_with_number_of_frames_stratified.csv')
df = get_df(df, 16, True)

In [ ]:
class_image_paths, end_idx = get_indices(df)

In [ ]:
valid_loader = get_loader(16, 4, end_idx, class_image_paths, valid_transform, tensor_transform, True)

# Load Model, change head, freeze body

In [ ]:
device = torch.device('cuda')

In [ ]:
cnn_encoder = ResCNNEncoder().to(device)
adaptive_pool = AdaptiveConcatPool2d()
cnn_encoder.resnet[8] = adaptive_pool

In [ ]:
for param in cnn_encoder.parameters():
    param.requires_grad = True
for param in cnn_encoder.resnet[8].parameters():
    param.requires_grad = True
for param in cnn_encoder.headbn1.parameters():
    param.requires_grad = True
for param in cnn_encoder.fc1.parameters():
    param.requires_grad = True

In [ ]:
rnn_decoder = DecoderRNNattention(batch_size=4).to(device)
for param in rnn_decoder.parameters():
    param.requires_grad = True

In [ ]:
crnn_params, cnn_encoder, rnn_decoder = parallelize_model(cnn_encoder, rnn_decoder)

## Torchsummary

In [ ]:
#summary(cnn_encoder.module, torch.zeros(2,16,3,576,704).cuda())

In [ ]:
#rnn_decoder.module

In [ ]:
#summary(rnn_decoder.module, torch.zeros(4,16,512).cuda()) ,  change the division by 2 to wok

# ##########

## LR Finder

In [ ]:
model = nn.Sequential(cnn_encoder,rnn_decoder) #!!!!!!!

In [ ]:
model

In [ ]:
torch.cuda.empty_cache()

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(crnn_params, lr=1e-7, weight_decay=1e-2)
lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
lr_finder.range_test(train_loader, end_lr=100, num_iter=200)
lr_finder.plot() # to inspect the loss-learning rate graph
lr_finder.reset() # to reset the model and optimizer to their initial state

In [ ]:
lr_finder.plot() 

In [ ]:
inputs = inputs.permute(0,2,1,3,4)

In [ ]:
out = cnn_encoder(inputs)

In [ ]:
out.shape

In [ ]:
out.size(0)

In [ ]:
inputs, classes = next(iter(valid_loader))

In [ ]:
inputs.shape

In [ ]:
inputs = inputs.to(device)
classes = classes.to(device)

In [ ]:
inputs.shape

In [ ]:
inputs = inputs.permute(0,2,1,3,4)

In [ ]:
outputs = rnn_decoder(cnn_encoder(inputs)) 

In [ ]:
outputs.shape

In [ ]:
h0, c0 = torch.from_numpy(np.zeros((2, 2, 512))).float(), torch.from_numpy(np.zeros((1, 64, 5))).float()

In [ ]:
h0.shape